# Angular Momentum

In [129]:
from sympy import MatrixSymbol, Matrix, sqrt, Rational, I # Sympy allows us to perform some algebraic manipulations for matrices.
from sympy.physics.quantum import hbar, Dagger
import numpy as np

In [130]:
def comm(a,b):
    return a*b - b*a

## Irreducible Representations

Here we build the irreducible representations of the angular momentum algebra

In [131]:
# Let us now define the m = -j, -j+1, ... , j-1, j
def m(l):
    if type(l) == int:
        ms = [-Rational(i,1) for i in np.arange(-l, l + 0.5, 1)]
        return ms
    elif type(l) == float and 2*l % 2 == 1:
        ms = [- Rational(i,1) for i in np.arange(-l, l + 0.5, 1)] # I add the 0.5 because python does not consider the final point
        return ms
    else: 
        raise TypeError("Only integers or half-integers are allowed") 

# Define the Kronecker delta function     
def kronecker(m,n):
    if m == n:
        return 1
    return 0

# Representation for the rotations
def J3(l):
    if type(l) == int:
        # print(f"j: {Rational(l,1)}")
        j3 = Matrix([[hbar * Rational(m(j)[p],1) * kronecker(p,q) for p in range(dim)] for q in range(dim)])
        return j3
    elif type(l) == float and 2*l % 2 == 1:
        # print(f"j: {Rational(l,1)}")
        j3 = Matrix([[hbar * Rational(- m + 1/2,1) * kronecker(m,n) for n in range(dim)] for m in range(dim)])
        return j3
    else: 
        raise TypeError("Only integers or half-integers are allowed")
        
def J_plus(l):
    # print(f"j: {Rational(l,1)}")
    j_plus = Matrix([[hbar * sqrt(Rational((j - m(j)[p])*(j + m(j)[p] + 1),1)) * kronecker(p,q + 1) for p in range(dim)] for q in range(dim)])
    return j_plus
        
def J_minus(l):
    # print(f"j: {Rational(l,1)}")
    j_minus = Matrix([[hbar * sqrt(Rational((j + m(j)[p])*(j - m(j)[p] + 1),1)) * kronecker(p,q - 1) for p in range(dim)] for q in range(dim)])
    return j_minus

## Labels (j, m)

In [109]:
'''
Some hard code first. 
Which j?
'''
j = 1/2 # I can use Rational(1,2) to have numbers as fractions
dim = int(2 * j + 1)

In [110]:
m(j)

[1/2, -1/2]

## Building the operators (J3, J_plus, J_minus)

In [33]:
# Let me define a generic matrix in this basis. There is no use for it now, but it is useful to see that the construction works.
# First we define the object
A = MatrixSymbol('A',dim,dim)
# Now we represent it
A = Matrix(A) 

In [111]:
J3(j)

Matrix([
[hbar/2,       0],
[     0, -hbar/2]])

In [113]:
J_plus(j)

Matrix([
[0, hbar],
[0,    0]])

In [112]:
J_minus(j)

Matrix([
[   0, 0],
[hbar, 0]])

## Checking the Lie Algebra 

Now, consistency requires that the the following commutation relations 
$$ [J_+, J_-] = 2 \hbar J_3 \qquad [J_3, J_\pm] = \pm \hbar J_\pm $$
are satisfied. 

In [85]:
comm(J3(j), J_minus(j)) + hbar * J_minus(j)

j: 1/2
j: 1/2
j: 1/2


Matrix([
[0, 0],
[0, 0]])

In [86]:
comm(J3(j), J_plus(j)) - hbar * J_plus(j)

j: 1/2
j: 1/2
j: 1/2


Matrix([
[0, 0],
[0, 0]])

In [87]:
comm(J_plus(j), J_minus(j)) - 2 * hbar * J3(j)

j: 1/2
j: 1/2
j: 1/2


Matrix([
[0, 0],
[0, 0]])

## Similarity transformation for the j=1 case

I would like to find a matrix $S$ such that the transformation 
$$ T_i = i S^\dagger J_i S$$
brings the matrices to the form of the SO(3) rotation generators. Observe that there is an extra factor of i. We define in this form to avoid terms of the form $\sqrt(i)$.

In [132]:
T1 = Matrix([[0,0,0],[0,0,1],[0,-1,0]])
T2 = Matrix([[0,0,-1],[0,0,0],[1,0,0]])
T3 = Matrix([[0,1,0],[-1,0,0],[0,0,0]])

In [133]:
T = [T1, T2, T3]

In [134]:
j = 1
dim = int(2 * j + 1)

In [135]:
m(j)

[1, 0, -1]

In [137]:
# This ugly symbol Jj is just to avoid confusion with the function J(j)
Jj1 = (J_plus(j) + J_minus(j)) / 2
Jj2 = (J_plus(j) - J_minus(j)) / (2 * I)
Jj3 = J3(j)

In [138]:
Jj = [Jj1, Jj2, Jj3]

In [170]:
S = Matrix([[1,-I,0],[0,0,-sqrt(2)],[-1,-I,0]]) / (sqrt(2 * hbar))

In [176]:
Transf = [I * Dagger(S) * Jj[a] * S - T[a] for a in range(len(m(j)))]

In [179]:
Transf[0]

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])